In [95]:
import fitz
import pandas as pd
from collections import Counter

In [20]:
pdf_address = "li.pdf"
#pdf_address = "judea.pdf"
pdf_doc = fitz.open(pdf_address)

In [21]:
page = pdf_doc.load_page(10)
page_text = page.get_text("text")
page_text

'1 Introduction\n3\nelectrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is one key point to facilitate functional imaging with\nalive animals. For example, with a cranial window over visual cortex of a transgenic\nmouse, it is able to image large scale of neu

In [22]:
page_blocks = page.get_text("blocks")
page_blocks


[(113.38600158691406,
  46.6934814453125,
  510.26995849609375,
  62.151676177978516,
  '1 Introduction\n3\n',
  0,
  0),
 (113.00399780273438,
  82.85345458984375,
  512.27734375,
  293.4193115234375,
  'electrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is

In [176]:
page_dicts = page.get_text("dict")

In [222]:
len(page_dicts["blocks"])

7

In [24]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

In [25]:
blocks = page.get_text("dict", flags=11)["blocks"]
page_font_sizes = []
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            print("")
            page_font_sizes.append(s["size"])
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            print("Text: '%s'" % s["text"])  # simple print of text
            print(font_properties)


Text: '1Introduction'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: '3'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'electrophysiologytocompensatethelowtemporalresolution.Figure2isanexam-'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'ple.Intra-cellularrecordingdirectlymeasuresneuronalactivities,usuallytakenas'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'groundtruth.Calciumimagingrecordsfluorescencesignalsasanindicatorofneu-'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'ronalspikingactivity,whichisindirect.Fluorescencesignalsandneuronalspikes'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'differfromeachother.Figure3showssingleactionpotentialanditsevokedcalcium'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000



In [26]:
page_font_sizes_counter = Counter(page_font_sizes)
page_font_sizes_counter

Counter({10.909099578857422: 29, 11.9552001953125: 1})

In [27]:
page_font_sizes_counter.most_common(1)[0]

(10.909099578857422, 29)

In [28]:
page_font_sizes_counter.most_common(1)

[(10.909099578857422, 29)]

In [29]:
#it can be modified to just iterate through the blocks instead of the spans

#develop it to pages with low amount of text. the bigest font size could be the title.

def find_titles(page):
    blocks = page.get_text("dict", flags=11)["blocks"]
    page_font_sizes = []
    page_font_names = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                #print("")
                page_font_sizes.append(s["size"])
                page_font_names.append(s["font"])
                
    page_font_sizes_counter = Counter(page_font_sizes)
    most_common_font_size, most_common_count = page_font_sizes_counter.most_common(1)[0]

    if most_common_count / len(page_font_sizes) > 0.5:
        title_font_sizes = [x for x in page_font_sizes if x > most_common_font_size]
    else:
        return None
    
    title_blocks = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                if s["size"] in title_font_sizes:
                    title_blocks.append(b)

    return title_blocks

In [30]:
a = find_titles(page)

In [31]:
a

[{'number': 3,
  'type': 0,
  'bbox': (113.38597106933594,
   527.8118286132812,
   406.64715576171875,
   544.9197387695312),
  'lines': [{'spans': [{'size': 11.9552001953125,
      'flags': 20,
      'font': 'LMSans10-Bold',
      'color': 0,
      'ascender': 1.1339999437332153,
      'descender': -0.296999990940094,
      'text': '1.1.1Intracellularcalciumdynamicsanditsmodeling',
      'origin': (113.38597106933594, 541.3690185546875),
      'bbox': (113.38597106933594,
       527.8118286132812,
       406.64715576171875,
       544.9197387695312)}],
    'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (113.38597106933594,
     527.8118286132812,
     406.64715576171875,
     544.9197387695312)}]}]

In [32]:
a[0]['bbox']

(113.38597106933594, 527.8118286132812, 406.64715576171875, 544.9197387695312)

In [154]:
title_text_dict = {}
page_blocks = page.get_text("blocks")
page_titles = find_titles(page)



In [155]:
test = fitz.open(pdf_address)
len(test)

67

In [117]:
def is_title(pblock, ptitles):
    if pblock is None:
        return False
    if not ptitles:
        return False
    for item in ptitles:
        if item['bbox'] == pblock[:4]:
            #print("bbox of title:", item['bbox'], "\n", "bbox of block:", pblock[:4])
            return True
    return False

In [107]:
def extract_image_from_page(pdf_document, page, page_number):
    image_list = page.get_images(full=True)
    if image_list is None:
        return None
    else:
        images_name = []
        info_names = []   #write decription about info of images in PyMuPDF.
        images_y0 = []
        for img_index, img_info in enumerate(image_list):
            xref = img_info[0]
            image_info_name = img_info[7]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page_number+1}_{img_index + 1}.png"
            y0 = page.get_image_bbox(img_info)[1]  
            images_name.append(image_filename)
            info_names.append(image_info_name)
            images_y0.append(y0)
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)
        return images_name, info_names, images_y0

In [108]:
a, b, c = extract_image_from_page(pdf_doc, page, 10)
c

[302.21148681640625, 303.86151123046875]

In [132]:
def make_master_list(pdf_doc):
    master_list = []

    # Iterate through each page
    for page_num in range(len(pdf_doc)):
        page = pdf_doc.load_page(page_num)

        # Extract text blocks from the page
        text_blocks = page.get_text("blocks")

        page_titles = find_titles(page)

        image_names, image_info_names, images_y0 = extract_image_from_page(pdf_doc, page, page_num)

        # Add page number to each block and append to master list
        for i, block in enumerate(text_blocks):
            # Create a dictionary for the block
            block_dict = {
                "page": page_num,
                "bbox": block[0:4],  # Position of the text block
                "text": block[4],   # Text content of the block
                "image": [],
                "title": is_title(block, page_titles)
            }
            
            next_block = text_blocks[i+1] if i < len(text_blocks) - 1 else None
            for j, image_y0 in enumerate(images_y0):
                if is_title(next_block, page_titles) and image_y0 < next_block[1]:
                    block_dict["image"].append(image_names[j])
    
            master_list.append(block_dict)

    return master_list

In [133]:
doc_list = make_master_list(pdf_doc)

parts_list = []
part_text = []

index = 0
start_page = 0
title = None
while index < len(doc_list):
    item = doc_list[index]
    next_item = doc_list[index+1] if index+1 < len(doc_list) else None
    #index starts from 0 but len() counts from 1
    if item['title'] == True:
        title = item["text"]
        start_page = item["page"]
        if next_item["title"]:
            end_page = next_item["page"]
            part_dict = {
                'title': title,
                'pages': f'{start_page + 1}-{end_page + 1}',
                'text': part_text,
                'images': item["image"]
            }
            parts_list.append(part_dict)
            part_text = []
    else:
        part_text.append(item["text"])
        if next_item is not None and next_item['title'] == True:
            end_page = next_item["page"]
            part_dict = {
                'title': title,
                'pages': f'{start_page + 1}-{end_page + 1}',
                'text': part_text,
                'images': item["image"]
            }
            parts_list.append(part_dict)
            part_text = []
    #print(index)
    index += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [134]:
parts_list

[{'title': None,
  'pages': '1-2',
  'text': ['Master Thesis\n',
   'Correlation analysis of signals\nobtained by calcium imaging in the\nprimary visual cortex of mice\n',
   'Xiuzhi Li\n',
   'Supervisor: Prof. Dr. Stefan Rotter\n',
   'M.Sc. Neuroscience\n',
   'University of Freiburg\n',
   'October 4th, 2022\n',
   '2\n'],
  'images': []},
 {'title': 'Abstract\n',
  'pages': '2-3',
  'text': ['Calcium imaging throws light on studying the activities of a large neuronal pop-\nulation.\nHowever, analyzing fluorescence signals is challenging.\nReconstructing\nspike trains from fluorescence signals by deconvolution is very sensitive to noise,\nnot stable, and linked with high computational costs. In this thesis, we propose and\ndemonstrate a new method to obtain spike train correlations by deconvolution of\nthe correlations of fluorescence signals, avoiding the reconstruction of spike trains\naltogether. Working with surrogate data generated by various point processes, it\nis shown that

In [135]:
#pd.set_option('display.max_colwidth', 100000)
#pd.set_option('display.max_rows', 100000)
pd.DataFrame(parts_list)

,title,pages,text,images
0,None,1-2,"[Master Thesis\n, Correlation analysis of signals\nobtained by calcium imaging in the\nprimary visual cortex of mice\n, Xiuzhi Li\n, Supervisor: Prof. Dr. Stefan Rotter\n, M.Sc. Neuroscience\n, University of Freiburg\n, October 4th, 2022\n, 2\n]",[]
1,Abstract\n,2-3,"[Calcium imaging throws light on studying the activities of a large neuronal pop-\nulation.\nHowever, analyzing fluorescence signals is challenging.\nReconstructing\nspike trains from fluorescence signals by deconvolution is very sensitive to noise,\nnot stable, and linked with high computational costs. In this thesis, we propose and\ndemonstrate a new method to obtain spike train correlations by deconvolution of\nthe correlations of fluorescence signals, avoiding the reconstruction of spike trains\naltogether. Working with surrogate data generated by various point processes, it\nis shown that the method captures the correlation structure faultlessly.\nThe in-\nferred cross-covariance is represented with experimental data obtained from calcium\nimaging of the mouse’s primary visual cortex, with which a connectivity inference\nis attempted. In addition, a validation process by applying the procedure of han-\ndling experimental data on surrogate data demonstrated that the cross-covariance\nof underlying spike trains could be estimated with high reliability. Finally, we show\nthat the mythology can be used to analyze the correlation of membrane potentials\nin a sub-threshold regime based on a shotnoise model of simple linear-modeled neu-\nrons.\n, Contents\n3\n]",[]
2,Contents\n,3-5,"[1\nIntroduction\n1\n, 1.1\nCalcium Imaging . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n2\n, 1.1.1\nIntracellular calcium dynamics and its modeling\n. . . . . . .\n3\n, 1.1.2\nCalcium indicators . . . . . . . . . . . . . . . . . . . . . . . .\n5\n, 1.1.3\nData analysis and its difficulties\n. . . . . . . . . . . . . . . .\n7\n, 1.2\nNeural coding: from firing rate to correlation . . . . . . . . . . . . .\n8\n, 1.2.1\nCorrelation quantification . . . . . . . . . . . . . . . . . . . .\n9\n, 1.3\nQuestion statement . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n10\n, 2\nMaterials and Methods\n11\n, 2.1\nSoftware and hardware . . . . . . . . . . . . . . . . . . . . . . . . . .\n11\n, 2.2\nFluorescence signal modeling\n. . . . . . . . . . . . . . . . . . . . . .\n11\n, 2.2.1\nPoint process . . . . . . . . . . . . . . . . . . . . . . . . . . .\n12\n, 2.2.2\nKernel . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n14\n, 2.2.3\nExact integration as fast convolution . . . . . . . . . . . . . .\n16\n, 2.3\nSignal characterization and correlation inference\n. . . . . . . . . . .\n17\n, 2.3.1\nOne signal . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n17\n, 2.3.2\nTwo signals . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n, 2.3.3\nMultiple signals . . . . . . . . . . . . . . . . . . . . . . . . . .\n24\n, 2.3.4\nModeling membrane potential . . . . . . . . . . . . . . . . . .\n24\n, 2.4\nPre-processing of experimental data\n. . . . . . . . . . . . . . . . . .\n25\n, 3\nResults\n27\n, 3.1\nSimulated fluorescence data . . . . . . . . . . . . . . . . . . . . . . .\n27\n, 3.1.1\nKernel selection . . . . . . . . . . . . . . . . . . . . . . . . . .\n27\n, 3.1.2\nIndependent Poisson process\n. . . . . . . . . . . . . . . . . .\n28\n, 3.1.3\nCorrelated Poisson process with double input . . . . . . . . .\n29\n, 3.1.4\nNon-stationary Poisson process . . . . . . . . . . . . . . . . .\n30\n, 3.1.5\nSIP\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n31\n, Contents\n4\n, 3.1.6\nMIP . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n32\n, 3.1.7\nHOC . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n34\n, 3.2\nExperimental data . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n35\n, 3.2.1\nData normalization . . . . . . . . . . . . . . . . . . . . . . . .\n36\n, 3

In [86]:
doc_list

[{'page': 0,
  'bbox': (268.5790100097656,
   78.97984313964844,
   355.0291748046875,
   99.16494750976562),
  'text': 'Master Thesis\n',
  'title': False},
 {'page': 0,
  'bbox': (131.86900329589844,
   152.36178588867188,
   491.89251708984375,
   220.10848999023438),
  'text': 'Correlation analysis of signals\nobtained by calcium imaging in the\nprimary visual cortex of mice\n',
  'title': False},
 {'page': 0,
  'bbox': (264.92498779296875,
   262.96148681640625,
   357.8518371582031,
   297.93609619140625),
  'text': 'Xiuzhi Li\n',
  'title': False},
 {'page': 0,
  'bbox': (163.44497680664062,
   339.4076843261719,
   460.19976806640625,
   368.5624694824219),
  'text': 'Supervisor: Prof. Dr. Stefan Rotter\n',
  'title': False},
 {'page': 0,
  'bbox': (251.68597412109375,
   578.2548217773438,
   371.92144775390625,
   598.4398803710938),
  'text': 'M.Sc. Neuroscience\n',
  'title': False},
 {'page': 0,
  'bbox': (244.1729736328125,
   599.7748413085938,
   379.4719543457031,
   6